### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import itertools
# Used for mathematical functions
import math             
import logging



# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
# import pyproj
# import shapely.geometry
# from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt
import tqdm


# Used for making the graph to visualize our problem
# import networkx as nx   

import plotly.express as px
from plotly.subplots import make_subplots

# OpenTNSim
import opentnsim

# turn on debug messages
logging.basicConfig(level=logging.DEBUG)
# turn off debug messages
#logging.basicConfig(level=logging.INFO)

### Create vessel - add VesselProperties and ConsumesEnergy mixins

In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.core.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [3]:
# Create a dict with all important settings
# NB: set both "v" and "P_tot_given" to None, as they will be set later

data_vessel = {"env": None,
               "name": 'Vessel M9',
               "route": None,
               "geometry": None,
               "v": None,  # m/s
               "type": None,
               "B": 11.45,
               "L": 135,
               "H_e": None, 
               "H_f": None, 
               "T": 2.75,
               "P_installed": 2200.0,   
               "P_tot_given": None, # kW
               "L_w": 3.0 ,
               "C_b":0.85, 
               "c_year":1990,
               "current_year":None
              }             

vessel = TransportResource(**data_vessel)

### Set vessel properties and sailing conditions ( input value )

In [4]:
V_s    = np.linspace(0.1, 8, 200)          # ship sailing speeds to water, (m/s)
h_0    = [10, 7.5, 5, 3.5]                 # water depths,(m)
c_year = [1970,1980,1990,2000,2010,2020]   # engine construction years

### prepare input matrix for calculation

In [5]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product(c_year, h_0, V_s))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"c_year": item[0], "h_0": item[1], "V_s": item[2]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.head()

,c_year,h_0,V_s
0,1970,10.0,0.100000
1,1970,10.0,0.139698
2,1970,10.0,0.179397
3,1970,10.0,0.219095
4,1970,10.0,0.258794


In [ ]:
# make a function that does all the work
def do_simulation(row):
    time.sleep(0.0001)
    f'running simulation for {row.c_year}, {row.h_0}, {row.V_s}'
    return row.c_year

# test the function on 1 row
row = work_df.iloc[0]
do_simulation(row)


In [ ]:
# apply (from map, reduce approach) the function to all the work
# show the rpogress
tqdm.tqdm.pandas(desc="running simulations")
# do the apply
results = work_df.progress_apply(do_simulation, axis=1)

In [ ]:
# show the results
results.head()

In [ ]:
work_df['V_s']

In [ ]:
work_df['h_0']

In [ ]:
work_df['c_year']

In [ ]:
energycalculation= opentnsim.core.ConsumesEnergy

for i in range(len(work_df['V_s'])):
        energycalculation.calculate_properties(vessel)
        energycalculation.calculate_frictional_resistance(vessel,work_df['V_s'][i],work_df['h_0'][i]) 
        energycalculation.calculate_viscous_resistance(vessel)
        energycalculation.calculate_appendage_resistance(vessel,work_df['V_s'][i])
        energycalculation.calculate_wave_resistance(vessel,work_df['V_s'][i],work_df['h_0'][i])
        energycalculation.calculate_residual_resistance(vessel,work_df['V_s'][i],work_df['h_0'][i])
        energycalculation.calculate_total_resistance(vessel,work_df['V_s'][i],work_df['h_0'][i])
        energycalculation.calculate_total_power_required(vessel)


DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 110.00930895945176 kW
DEBUG:opentnsim.core:The actual total power given is 110.00930895945176 kW
DEBUG:opentnsim.core:The partial load is 0.05000423134520535
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 110.02427427747801 kW
DEBUG:opentnsim.core:The actual total power given is 110.02427427747801 kW
DEBUG:opentnsim.core:The partial load is 0.050011033762490005
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 110.04977153402406 kW
DEBUG:opentnsim.core:The act

DEBUG:opentnsim.core:The total power required is 116.48897395363956 kW
DEBUG:opentnsim.core:The actual total power given is 116.48897395363956 kW
DEBUG:opentnsim.core:The partial load is 0.05294953361529071
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 117.28160929803173 kW
DEBUG:opentnsim.core:The actual total power given is 117.28160929803173 kW
DEBUG:opentnsim.core:The partial load is 0.05330982240819624
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 118.13488746614546 kW
DEBUG:opentnsim.core:The actual total power given is 118.13488746614546 kW
DEBUG:opentnsim.core:The partial load is 0.05369767612097521
DEBUG:opentnsim.core:The frictional_resistance is <boun

DEBUG:opentnsim.core:The partial load is 0.06869235390497501
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 153.73727033809186 kW
DEBUG:opentnsim.core:The actual total power given is 153.73727033809186 kW
DEBUG:opentnsim.core:The partial load is 0.0698805774264054
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 156.4619467967735 kW
DEBUG:opentnsim.core:The actual total power given is 156.4619467967735 kW
DEBUG:opentnsim.core:The partial load is 0.07111906672580613
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power requi

DEBUG:opentnsim.core:The total power required is 247.3152477872976 kW
DEBUG:opentnsim.core:The actual total power given is 247.3152477872976 kW
DEBUG:opentnsim.core:The partial load is 0.11241602172149891
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 253.78479702047883 kW
DEBUG:opentnsim.core:The actual total power given is 253.78479702047883 kW
DEBUG:opentnsim.core:The partial load is 0.11535672591839947
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 260.5142624770332 kW
DEBUG:opentnsim.core:The actual total power given is 260.5142624770332 kW
DEBUG:opentnsim.core:The partial load is 0.1184155738531969
DEBUG:opentnsim.core:The frictional_resistance is <bound met

DEBUG:opentnsim.core:The partial load is 0.21273644044285453
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 483.05403932350004 kW
DEBUG:opentnsim.core:The actual total power given is 483.05403932350004 kW
DEBUG:opentnsim.core:The partial load is 0.2195700178743182
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 498.6338203726042 kW
DEBUG:opentnsim.core:The actual total power given is 498.6338203726042 kW
DEBUG:opentnsim.core:The partial load is 0.2266517365330019
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power requir

...

DEBUG:opentnsim.core:The partial load is 0.06484966570213822
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 144.9352921755413 kW
DEBUG:opentnsim.core:The actual total power given is 144.9352921755413 kW
DEBUG:opentnsim.core:The partial load is 0.06587967826160968
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 147.30332394828963 kW
DEBUG:opentnsim.core:The actual total power given is 147.30332394828963 kW
DEBUG:opentnsim.core:The partial load is 0.06695605634013164
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power requ

DEBUG:opentnsim.core:The total power required is 227.2144203246943 kW
DEBUG:opentnsim.core:The actual total power given is 227.2144203246943 kW
DEBUG:opentnsim.core:The partial load is 0.10327928196577013
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 232.9118965431175 kW
DEBUG:opentnsim.core:The actual total power given is 232.9118965431175 kW
DEBUG:opentnsim.core:The partial load is 0.10586904388323523
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 238.8388043224855 kW
DEBUG:opentnsim.core:The actual total power given is 238.8388043224855 kW
DEBUG:opentnsim.core:The partial load is 0.10856309287385704
DEBUG:opentnsim.core:The frictional_resistance is <bound meth

DEBUG:opentnsim.core:The partial load is 0.193635369443199
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 438.9238179095716 kW
DEBUG:opentnsim.core:The actual total power given is 438.9238179095716 kW
DEBUG:opentnsim.core:The partial load is 0.19951082632253253
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 452.4175338500619 kW
DEBUG:opentnsim.core:The actual total power given is 452.4175338500619 kW
DEBUG:opentnsim.core:The partial load is 0.20564433356820994
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required

DEBUG:opentnsim.core:The total power required is 934.0906296333621 kW
DEBUG:opentnsim.core:The actual total power given is 934.0906296333621 kW
DEBUG:opentnsim.core:The partial load is 0.42458664983334643
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 970.5785027582085 kW
DEBUG:opentnsim.core:The actual total power given is 970.5785027582085 kW
DEBUG:opentnsim.core:The partial load is 0.4411720467082766
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 1008.753865262818 kW
DEBUG:opentnsim.core:The actual total power given is 1008.753865262818 kW
DEBUG:opentnsim.core:The partial load is 0.4585244842103718
DEBUG:opentnsim.core:The frictional_resistance is <bound method

DEBUG:opentnsim.core:The partial load is 1
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 2326.1339955470266 kW
DEBUG:opentnsim.core:The actual total power given is 2200.0 kW
DEBUG:opentnsim.core:The partial load is 1
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 2419.156444946996 kW
DEBUG:opentnsim.core:The actual total power given is 2200.0 kW
DEBUG:opentnsim.core:The partial load is 1
DEBUG:opentnsim.core:The frictional_resistance is <bound method ConsumesEnergy.calculate_frictional_resistance of <__main__.Vessel object at 0x00000112FC2101F0>>
DEBUG:opentnsim.core:The total power required is 2515.3716728435243 kW
DEBUG:opentnsim.core:The actual total power giv

### Run OpenTNSim to Calculate resistance, required power and emissions 
Now we can define the run. First, we make an environment and add the graph to the environment. Then we add one vessel, to which we will append the environment and the above route. Lastly, we give the vessel the process of moving from the origin to the destination of the defined path and subsequently run the model.

In [ ]:
# create empty plot data
plot_data = {}

    
        # create an EnergyCalculation object and perform energy consumption calculation

        frictional_resistance = opentnsim.core.ConsumesEnergy.calculate_frictional_resistance(row.V_s,row.h_0)    
        wave_resistance = opentnsim.core.ConsumesEnergy.calculate_wave_resistance
        viscous_resistance = opentnsim.core.ConsumesEnergy.calculate_viscous_resistance
        appendage_resistance = opentnsim.core.ConsumesEnergy.calculate_appendage_resistance
        residual_resistance = opentnsim.core.ConsumesEnergy.calculate_residual_resistance
        total_resistance = opentnsim.energy.calculate_total_resistance(v=V_s, h_0).append()

        total_power_required = opentnsim.energy.calculate_total_power_required().append()

        emission_g_m_CO2_speed=opentnsim.energy.EnergyCalculation.
        emission_g_m_PM10_speed=
        emission_g_m_NOX_speed=
        emission_g_m_CO2_engine_age=opentnsim.energy.EnergyCalculation.
        emission_g_m_PM10_engine_age=
        emission_g_m_NOX_engine_age= 
    
    
    # create dataframe from energy calculation computation
    df = pd.DataFrame.from_dict(energycalculation.energy_use)    

In [ ]:
plot_data = {}

### Figures output

### Figure 3: Components of the total resistance

In [ ]:
# resistance for water depth h2 = 7.5 m
R_f_2 = frictional_resistance(h2)[0]
R_APP_2 = frictional_resistance(h2)[1]

plt.figure(figsize=(6,4))
plt.plot(V_0_km, R_tot2,color='#0165fc', linewidth = 2.5)       # RT, total resistance of the ship,[kN] 
plt.plot(V_0_km, one_k1*R_f_2,color='#ff9408', linewidth = 1.7) # Rf(1+k1), frictional resistance including the hull form factor
plt.plot(V_0_km, R_APP_2,'-.', color='#76cd26',linewidth = 2)   # Rapp, appendage resistance
plt.plot(V_0_km, R_W_2, 'r--', linewidth = 2)                   # Rw, wave resistance
plt.plot(V_0_km, R_TR_2 + R_A_2,':',color='#aa23ff', linewidth = 2) # Rres, residual resistance
plt.grid()
plt.xlim(0, 18)
plt.ylim(0, 130)
plt.legend(['$R_T$', '$R_f$$\ $(1+$k_1$)', '$R_{app}$', '$R_W$','$R_{res}$'])
plt.xlabel('Relative vessel speed (km/h)',fontsize=10,weight='bold',labelpad=3)
plt.ylabel('Resistance (kN)',fontsize=10,weight='bold',labelpad=3)
plt.xticks( weight = 'bold')
plt.yticks( weight = 'bold')


#plt.savefig('resistance components.pdf', dpi=600, format='pdf') 
plt.savefig('PartIV-05-04.eps', dpi=600, format='eps') 
#plt.title('resistance components (L = 135 m, B = 11.45 m, T = 2.75 m, h = 7.5 m)')

### Figure 5: Total required power

In [ ]:
plt.figure(figsize = (6,4))
plt.plot(V_0_km, P_tot1, color='navy',linewidth = 2.0)
plt.plot(V_0_km, P_tot2,'-.', color='royalblue',linewidth = 2.0)
plt.plot(V_0_km, P_tot3,'--', color='lightcoral',linewidth = 2.0)
plt.plot(V_0_km, P_tot4,':', color='indianred',linewidth = 2.0)

plt.grid()
plt.xlim(10, 18)
plt.ylim(0, 2000)
plt.legend(['$h_0$ = 10 m', '$h_0$ = 7.5 m', '$h_0$ = 5 m', '$h_0$ = 3.5 m'])

plt.xlabel('Relative vessel speed (km/h)',fontsize=10,weight='bold',labelpad=3)
plt.ylabel('Power (kW)',fontsize=10,weight='bold',labelpad=3)
#plt.title('total power required versus speed with diff water depths (type = M9, L = 135 m, B = 11.45 m, T = 2.75 m)')
plt.xticks( weight = 'bold')
plt.yticks( weight = 'bold')
plt.savefig('total power required versus speed with diff water depths.pdf', dpi=600, format='pdf') 


### Figure 7: validation_total power required versus speed with diff water depths (add DST results to Fig.5)

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline

plt.figure(figsize = (6,4))
V_DST_1=np.array([10,11,12,13,14,15,16,17,18]) # deep water
P_DST_1=np.array([180,240,300,400,500,640,800,1000,1200])
V_DST_2=np.array([10,11,12,13,14,15,16,17,18]) # h=7.5 m
P_DST_2=np.array([230,300,380,490,600,780,1000,1300,1600])
V_DST_3=np.array([10,11,12,13,14,15,15.65,16,16.5,16.7]) # h=5 m
P_DST_3=np.array([275,360,470,600,800,1000,1200,1400,1800,2005])
V_DST_4=np.array([10,10.5,11,11.5,12,12.5,13]) # h=3.5 m
P_DST_4=np.array([350,450,535,685,1000,1800,3000])
#define spline, make the lines smooth
V_DST_1_new = np.linspace(V_DST_1.min(), V_DST_1.max(), 200) 
V_DST_2_new = np.linspace(V_DST_2.min(), V_DST_2.max(), 200) 
V_DST_3_new = np.linspace(V_DST_3.min(), V_DST_3.max(), 200) 
V_DST_4_new = np.linspace(V_DST_4.min(), V_DST_4.max(), 200) 

spl1= make_interp_spline(V_DST_1,P_DST_1, k=3)
P_DST_1_new = spl1(V_DST_1_new)
spl2= make_interp_spline(V_DST_2,P_DST_2, k=3)
P_DST_2_new = spl2(V_DST_2_new)
spl3= make_interp_spline(V_DST_3,P_DST_3, k=3)
P_DST_3_new = spl3(V_DST_3_new)
spl4= make_interp_spline(V_DST_4,P_DST_4, k=3)
P_DST_4_new = spl4(V_DST_4_new)
                 
plt.plot(V_DST_1_new,P_DST_1_new,color='navy',linewidth = 3.0)
plt.plot(V_DST_2_new,P_DST_2_new,color='royalblue',linewidth = 3.0)
plt.plot(V_DST_3_new,P_DST_3_new,color='lightcoral',linewidth = 3.0)
plt.plot(V_DST_4_new,P_DST_4_new,color='indianred',linewidth = 3.0)


plt.plot(V_0_km, P_tot1,'-', color='navy',linewidth = 2.0)
plt.plot(V_0_km, P_tot2,'-.', color='royalblue',linewidth = 2.0)
plt.plot(V_0_km, P_tot3,'--', color='lightcoral',linewidth = 2.0)
plt.plot(V_0_km, P_tot4,':', color='indianred',linewidth = 2.0)

plt.grid()
plt.xlim(10, 18)
plt.ylim(0, 2000)
plt.legend(['DST, deep water', 'DST, $h_0$ = 7.5 m', 'DST, $h_0$ = 5 m', 'DST, $h_0$ = 3.5 m',
            'Own, $h_0$ = 10 m', 'Own, $h_0$ = 7.5 m', 'Own, $h_0$ = 5 m', 'Own, $h_0$ = 3.5 m'],loc=(1.04,0.2))

plt.xlabel('Relative vessel speed (km/h)',fontsize=10,weight='bold',labelpad=3)
plt.ylabel('Power (kW)',fontsize=10,weight='bold',labelpad=3)
#plt.title('total power required versus speed with diff water depths (type = M9, L = 135 m, B = 11.45 m, T = 2.75 m)')
plt.xticks( weight = 'bold')
plt.yticks( weight = 'bold')
plt.savefig('validation_total power required versus speed with diff water depths.pdf',bbox_inches = 'tight',dpi=600, format='pdf') 

### Figure 8: Emissions as a function of vessel speed: emission rates in g/km versus speed for different water depths

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 3),constrained_layout=True)
fontsize=10

ax=plt.subplot(131)
plt.plot(V_0_km, emission_g_m_CO2_1, 'yellow',linewidth = 2.0) # CO2 emission rates for water depth h1=10 m
plt.plot(V_0_km, emission_g_m_CO2_2, 'orange',linewidth = 2.0) # CO2 emission rates for water depth h2=7.5 m
plt.plot(V_0_km, emission_g_m_CO2_3, 'red',linewidth = 2.0)    # CO2 emission rates for water depth h3=5 m
plt.plot(V_0_km[V_0_km<15.6], emission_g_m_CO2_4[V_0_km<15.6], 'darkred',linewidth = 2.0) # for water depth h4=3.5 m, speed limited to 15.6 km/h
plt.legend(['$h_0$ = 10 m', '$h_0$ = 7.5 m', '$h_0$ = 5 m', '$h_0$ = 3.5 m'],loc='upper center')
plt.grid()
plt.xlim(0, 18)  #  set vessel speed range (km/h) for inland shipping
plt.ylim(0, 300000)
plt.xlabel('Relative vessel speed (km/h)',fontsize=10,weight='bold',labelpad=3)
plt.ylabel('CO2 emission rate (g/km)',fontsize=10,weight='bold',labelpad=3)
plt.xticks([0,5,10,15,18], weight = 'bold')
plt.yticks( weight = 'bold')

ax=plt.subplot(132)
plt.plot(V_0_km, emission_g_m_PM10_1, 'paleturquoise',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_PM10_2, 'mediumspringgreen',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_PM10_3, 'mediumseagreen',linewidth = 2.0)
plt.plot(V_0_km[V_0_km<15.6], emission_g_m_PM10_4[V_0_km<15.6], 'darkgreen',linewidth = 2.0)
plt.legend(['$h_0$ = 10 m', '$h_0$ = 7.5 m', '$h_0$ = 5 m', '$h_0$ = 3.5 m'],loc='upper center')
plt.grid()
plt.xlim(0, 18)
plt.ylim(0, 350)
plt.xlabel('Relative vessel speed (km/h)',fontsize=10,weight='bold',labelpad=3)
plt.ylabel('PM10 emission rate (g/km)',fontsize=10,weight='bold',labelpad=3)
plt.xticks([0,5,10,15,18], weight = 'bold')
plt.yticks( weight = 'bold')

ax=plt.subplot(133)
plt.plot(V_0_km, emission_g_m_NOX_1, 'pink',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_NOX_2, 'hotpink',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_NOX_3, 'deeppink',linewidth = 2.0)
plt.plot(V_0_km[V_0_km<15.6], emission_g_m_NOX_4[V_0_km<15.6], 'purple',linewidth = 2.0)
plt.legend(['$h_0$ = 10 m', '$h_0$ = 7.5 m', '$h_0$ = 5 m', '$h_0$ = 3.5 m'],loc='upper center')
plt.grid()
plt.xlim(0, 18)
plt.ylim(0, 6500)
plt.xlabel('Relative vessel speed (km/h)',fontsize=10, weight='bold',labelpad=3)
plt.ylabel('NOx emission rate (g/km)',fontsize=10,weight='bold',labelpad=3)
plt.xticks([0,5,10,15,18], weight = 'bold')
plt.yticks( weight = 'bold')

#plt.savefig('emission rates versus speed with diff water depth.pdf', dpi=600, format='pdf')  
plt.savefig('PartIV-05-09.eps', dpi=600, format='eps') 

### Figure 9: Emissions as a function of engine age: emission rates in g/km versus speed for different engine ages


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 3),constrained_layout=True)
fontsize=10

ax=plt.subplot(131)
plt.plot(V_0_km, emission_g_m_CO2_11, 'darkred',linewidth = 2.0)   # CO2 emission rates for engine construction year 1970
plt.plot(V_0_km, emission_g_m_CO2_22, 'red',linewidth = 2.0)       # CO2 emission rates for engine construction year 1980
plt.plot(V_0_km, emission_g_m_CO2_33, 'darkorange',linewidth = 2.0)# CO2 emission rates for engine construction year 1990
plt.plot(V_0_km, emission_g_m_CO2_44, 'orange',linewidth = 2.0)    # CO2 emission rates for engine construction year 2000
plt.plot(V_0_km, emission_g_m_CO2_55, 'gold',linewidth = 2.0)      # CO2 emission rates for engine construction year 2010
plt.plot(V_0_km, emission_g_m_CO2_66, 'yellow',linewidth = 2.0)    # CO2 emission rates for engine construction year 2020
plt.legend(['$C_{year}$ = 1970', '$C_{year}$ = 1980', '$C_{year}$ = 1990', '$C_{year}$ = 2000', 
            '$C_{year}$ = 2010', '$C_{year}$ = 2020'],loc='upper center')
plt.grid()
plt.xlim(0, 18)
plt.ylim(0, 100000)
plt.xlabel('Relative vessel speed (km/h)',fontsize=10, weight='bold',labelpad=3)
plt.ylabel('CO2 emission rate (g/km)',fontsize=10,weight='bold',labelpad=3)
plt.xticks([0,5,10,15,18],weight = 'bold')
plt.yticks( weight = 'bold')

ax=plt.subplot(132)
plt.plot(V_0_km, emission_g_m_PM10_11, 'darkgreen',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_PM10_22, 'seagreen',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_PM10_33, 'mediumseagreen',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_PM10_44, 'mediumspringgreen')
plt.plot(V_0_km, emission_g_m_PM10_55, 'aquamarine',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_PM10_66, 'paleturquoise',linewidth = 2.0)
plt.legend(['$C_{year}$ = 1970', '$C_{year}$ = 1980', '$C_{year}$ = 1990', '$C_{year}$ = 2000', 
            '$C_{year}$ = 2010', '$C_{year}$ = 2020'],loc='upper center')
plt.grid()
plt.xlim(0, 18)
plt.ylim(0, 100)
plt.xlabel('Relative vessel speed (km/h)',fontsize=10, weight='bold',labelpad=3)
plt.ylabel('PM10 emission rate (g/km)',fontsize=10,weight='bold',labelpad=3)
plt.xticks([0,5,10,15,18], weight = 'bold')
plt.yticks( weight = 'bold')

ax=plt.subplot(133)
plt.plot(V_0_km, emission_g_m_NOX_11, 'purple',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_NOX_22, 'darkmagenta',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_NOX_33, 'orchid',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_NOX_44, 'deeppink',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_NOX_55, 'hotpink',linewidth = 2.0)
plt.plot(V_0_km, emission_g_m_NOX_66, 'pink',linewidth = 2.0)
plt.legend(['$C_{year}$ = 1970', '$C_{year}$ = 1980', '$C_{year}$ = 1990', '$C_{year}$ = 2000',
            '$C_{year}$ = 2010', '$C_{year}$ = 2020'],loc='upper center')
plt.grid()
plt.xlim(0, 18)
plt.ylim(0, 1600)
plt.xlabel('Relative vessel speed (km/h)',fontsize=10, weight='bold',labelpad=3)
plt.ylabel('NOx emission rate (g/km)',fontsize=10,weight='bold',labelpad=3)
plt.xticks([0,5,10,15,18], weight = 'bold')
plt.yticks( weight = 'bold')

#plt.savefig('emission rates versus speed with diff engine ages.pdf', dpi=600, format='pdf')
plt.savefig('PartIV-05-10.eps', dpi=600, format='eps')
#fig.suptitle('Emission rate versus velocity, for different engine ages  \n (type = M9, L = 135 m, B = 11.45 m, T = 2.75 m, h = 10 m)')